In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from scipy import stats
from os import listdir
from os.path import isfile, join
import re
import sys

##### this week we will make simple time-serie prediction model. Before dealing with this problem we are going to create .pickle files with  aggregated data coordinates: regions-hours. Numbers in table cells are the amount of taxi trips. Do all this stuff for six month of 2014. Later in this project we will work with data collected for 2 year period, but now we pick just half of the 2014 year

###### all .csv files were gathered from source specified in week_1 task or notebook related to first week

In [3]:
mypath = "H:/Yandex machine learning/finall course coursera/csv_yellow_cab_2014"

In [5]:
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['yellow_tripdata_2014-12.csv', 'yellow_tripdata_2014-11.csv', 'yellow_tripdata_2014-10.csv', 'yellow_tripdata_2014-09.csv', 'yellow_tripdata_2014-08.csv', 'yellow_tripdata_2014-07.csv', 'yellow_tripdata_2014-06.csv', 'yellow_tripdata_2014-05.csv']


In [6]:
NY_long_west = -74.25559
NY_long_east = -73.70001
NY_latt_south = 40.49612
NY_latt_north = 40.91553

In [7]:
def region_counter(longitude, latitude):
    long_num = np.ceil((longitude - NY_long_west)*50.0/(NY_long_east - NY_long_west))
    lat_num = np.ceil((latitude - NY_latt_south)*50.0/(NY_latt_north - NY_latt_south))
    
    return (long_num - 1) * 50.0 + lat_num

In [8]:
for file in onlyfiles[7:]:
    print(file)
    
    ''' for modified csv file '''
    #taxi = pd.read_csv(mypath + "/" + file, sep='\t', usecols=["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", 
    #                                                 "passenger_count", "trip_distance","pickup_longitude", "pickup_latitude"])
    if "2014" in re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", file):
        taxi = pd.read_csv(mypath + "/" + file, usecols=[" pickup_datetime", " dropoff_datetime", 
                                                     " passenger_count", " trip_distance", " pickup_longitude", " pickup_latitude"])
        taxi.rename(index=str, columns={" pickup_datetime": "tpep_pickup_datetime", 
                                        " dropoff_datetime": "tpep_dropoff_datetime", " passenger_count": "passenger_count", 
                                        " trip_distance": "trip_distance", " pickup_longitude": "pickup_longitude", 
                                        " pickup_latitude": "pickup_latitude"}, inplace=True)
    else:
        taxi = pd.read_csv(mypath + "/" + file, usecols=["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", 
                                                     "passenger_count", "trip_distance","pickup_longitude", "pickup_latitude"])
        
    print(file, " start num of trips: ", taxi.shape[0])
    taxi.drop(taxi[taxi.passenger_count == 0].index, inplace=True)
    taxi.drop(taxi[taxi.trip_distance == 0].index, inplace=True)
    taxi.drop(taxi[taxi.tpep_pickup_datetime == taxi.tpep_dropoff_datetime].index, inplace=True)
    taxi.drop(taxi[taxi.pickup_longitude < -74.25559].index, inplace=True)
    taxi.drop(taxi[taxi.pickup_longitude > -73.70001].index, inplace=True)
    taxi.drop(taxi[taxi.pickup_latitude < 40.49612].index, inplace=True)
    taxi.drop(taxi[taxi.pickup_latitude > 40.91553].index, inplace=True)
    columns = ['passenger_count','trip_distance', 'pickup_longitude', 'pickup_latitude']
    null_val = taxi[columns].isnull().values.any()
    print(file, " final num of trips: ", taxi.shape[0])
    print(file, " null values: ", null_val)
    if null_val:
        taxi = taxi[np.isfinite(df[columns])]
        print(file, " after droping NaNs num of trips: ", taxi.shape[0])
    taxi["region"] = region_counter(taxi.pickup_longitude, taxi.pickup_latitude)
    taxi["hour_statistic"] = pd.DatetimeIndex(taxi.tpep_pickup_datetime).map(lambda x: x.replace(minute=0, second=0))
    hours_in_month = np.sort(np.unique(taxi.hour_statistic)).astype(np.int64)
    hours_in_month = np.hstack((hours_in_month, hours_in_month[-1] + 3600000000000)) #add last hour
    
    region_bined_stat = stats.binned_statistic_2d(taxi.region, taxi.hour_statistic.astype(np.int64), 
                                            "None", 'count', bins=[np.arange(1, 2502), hours_in_month])
    
    file_name = "reg_bin_stat_" + re.findall("\d+\-\d+", file)[0] + ".pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(region_bined_stat.statistic, f)
    
    # These are the usual ipython objects, including this one you are creating
    ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

    # Get a sorted list of the objects and their sizes
    #print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True))
    
    del(taxi)  
    del(region_bined_stat)

yellow_tripdata_2014-05.csv
yellow_tripdata_2014-05.csv  start num of trips:  14774041
yellow_tripdata_2014-05.csv  final num of trips:  14344559
yellow_tripdata_2014-05.csv  null values:  False


##### this code helps to reduce size of original .csv files. Original files have quite big size (2 Gb each)and it may case problems on machines with low RAM.  We just do:int64=>int, float64=>float transformation

In [19]:
taxi = pd.read_csv(mypath + "/" + 'yellow_tripdata_2015-06.csv')

In [20]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [21]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12324935 entries, 0 to 12324934
Data columns (total 19 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RateCodeID               int64
store_and_fwd_flag       object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.7+ GB


In [22]:
list(taxi.columns)

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'RateCodeID',
 'store_and_fwd_flag',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount']

In [23]:
print(mem_usage(taxi))

4019.86 MB


In [24]:
taxi.loc[:, taxi.dtypes == np.int64] = taxi.loc[:, taxi.dtypes == np.int64].apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(taxi))

In [26]:
taxi.loc[:, taxi.dtypes == np.float64] = taxi.loc[:, taxi.dtypes == np.float64].apply(pd.to_numeric,downcast='float')
print(mem_usage(taxi))

In [28]:
taxi.to_csv(mypath + "/" + "yellow_tripdata_2015-06_edit.csv", sep='\t', columns=list(taxi.columns))